# Load Data

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [49]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
#md = md[md["notebook_name"] == "diff_rounds_fixed_dist"]
max_distance = int(max(md.distance))
max_distance = 55
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]
md = md.sort_values(by=['rounds'], ascending=False)
max_rounds = '35'
md = md[md["rounds"] == max_rounds]

md = md[:]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
892,2023-11-03 08:54:35.373280+01:00,diff_rounds_fixed_dist,ibm_sherbrooke,cn2aees3r3vg008f52vg,[],330.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,55.0,35,1,_is_hex=True,NaN
891,2023-11-03 08:54:12.891496+01:00,diff_rounds_fixed_dist,ibm_sherbrooke,cn2ae91p1am0008q77k0,[],330.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,55.0,35,0,_is_hex=True,NaN


In [54]:
from soft_info import get_repcode_layout, get_KDEs

memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memory = memories["mmr_log_0"][:]

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)
kde_dict, scaler_dict = get_KDEs(provider, DEVICE, layout, bandwidths=0.2, plot=False)

# Decoding

In [55]:
import stim
import pymatching

d = max_distance
T = int(max_rounds)

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=T,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)


In [57]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

#actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:]):
    i += 1
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    count_key = next(iter(counts.keys()))
    
    actual_observables = [(int(count_key[0])+0)%2]
    soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    #reweight_edges_to_one(matching)

    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    #print("actual_observables:", actual_observables)
    #print("predicted_observables:", predicted_observables)
    print("num_errors:", num_errors)

    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)
        print("actual_observables:", actual_observables)
        print("predicted_observables:", predicted_observables)

    
    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        
    if VERBOSE:
        draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))


    

print("Num errors:", num_errors)

  1%|          | 4/330 [00:16<21:52,  4.03s/it]

num_errors: 1


 33%|███▎      | 110/330 [07:29<14:55,  4.07s/it]

num_errors: 2


100%|██████████| 330/330 [22:30<00:00,  4.09s/it]

Num errors: 2


In [58]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

#actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:]):
    i += 1
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    count_key = next(iter(counts.keys()))
    
    actual_observables = [(int(count_key[0])+0)%2]
    #soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    reweight_edges_to_one(matching)

    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    #print("actual_observables:", actual_observables)
    #print("predicted_observables:", predicted_observables)
    print("num_errors:", num_errors)

    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)
        print("actual_observables:", actual_observables)
        print("predicted_observables:", predicted_observables)

    
    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        
    if VERBOSE:
        draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))


    

print("Num errors:", num_errors)

  1%|          | 3/330 [00:04<07:49,  1.44s/it]

num_errors: 1


 32%|███▏      | 107/330 [02:33<05:22,  1.45s/it]

num_errors: 2


 43%|████▎     | 142/330 [03:23<04:27,  1.42s/it]

num_errors: 3


 75%|███████▌  | 248/330 [05:55<01:59,  1.46s/it]

num_errors: 4


100%|██████████| 330/330 [07:54<00:00,  1.44s/it]

Num errors: 4
